### Data Preprocessing 

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_colwidth', 500) # Sets max column width to 500 characters
pd.set_option('display.max_columns', None) # Display all columns
pd.set_option('display.width', 1000) # Set display width

# --- Data Loading from CSV Files ---

VMobile_data_df = pd.read_csv('Data/VMobile_subscribers.csv', sep=';')
BlueMobile_data_df = pd.read_csv('Data/VMobile_subscribers_bluemobile.csv', sep=';')
ArrowMobile_data_df = pd.read_csv('Data/VMobile_subscribers_arrowmobile.csv', sep=';')
City_lookup_df = pd.read_csv('Data/VMobile_city_lookup.csv', sep=';') # Corrected sep for City_lookup
usage_event_lookup_df = pd.read_csv('Data/VMobile_usage_event_lookup.csv', sep=';')
usage_record_df = pd.read_csv('Data/VMobile_usage_records.csv', sep=';') # Corrected sep for usage_records
# Assuming VMobile_usage_event_lookup_wk2.csv is present and has the same format
usage_event_lookup_wk2_df = pd.read_csv('Data/VMobile_usage_records_week_2.csv', sep=';')

In [3]:
# --- V Mobile Data Processing ---
vmobile_df_processed = VMobile_data_df.copy()
vmobile_df_processed.columns = [col.strip() for col in vmobile_df_processed.columns]
vmobile_df_processed = vmobile_df_processed.rename(columns={
    'Cell Number': 'Cell_Phone_Number',
    'SIM Activation Date': 'SIM_Activation_Date',
    'First Name': 'First_Name',
    'Last Name': 'Last_Name',
    'Birthday': 'Date_Of_Birth',
    'Location': 'Region'
})
vmobile_df_processed['Source_System_Name'] = 'V Mobile'

# Clean phone numbers
vmobile_df_processed['Cell_Phone_Number'] = (
    vmobile_df_processed['Cell_Phone_Number']
    .astype(str)
    .str.replace('+', '', regex=False)
    .str.strip()
)

# Parse dates
vmobile_df_processed['SIM_Activation_Date'] = pd.to_datetime(
    vmobile_df_processed['SIM_Activation_Date'].astype(str).str.strip(),
    format='%d %m %Y',
    errors='coerce'
)
vmobile_df_processed['Date_Of_Birth'] = pd.to_datetime(
    vmobile_df_processed['Date_Of_Birth'].astype(str).str.strip(),
    format='%d %m %Y',
    errors='coerce'
)


In [4]:


# --- BlueMobile Data Processing ---
bluemobile_df_processed = BlueMobile_data_df.copy()
bluemobile_df_processed.columns = [col.strip() for col in bluemobile_df_processed.columns]
bluemobile_df_processed = bluemobile_df_processed.rename(columns={
    'Cell': 'Cell_Phone_Number',
    'Name': 'First_Name',
    'Surname': 'Last_Name',
    'City': 'Region',
    'Activate': 'SIM_Activation_Date',
    'Date': 'Date_Of_Birth'
})
bluemobile_df_processed['Source_System_Name'] = 'BlueMobile'

# Clean phone numbers (prepend 27 if missing)
bluemobile_df_processed['Cell_Phone_Number'] = (
    bluemobile_df_processed['Cell_Phone_Number']
    .astype(str)
    .str.replace('+', '', regex=False)
    .str.strip()
    .apply(lambda x: '27' + x if not x.startswith('27') else x)
)

# Parse dates
bluemobile_df_processed['SIM_Activation_Date'] = pd.to_datetime(
    bluemobile_df_processed['SIM_Activation_Date'].astype(str).str.strip(),
    format='%d %m %Y',
    errors='coerce'
)
bluemobile_df_processed['Date_Of_Birth'] = pd.to_datetime(
    bluemobile_df_processed['Date_Of_Birth'].astype(str).str.strip(),
    format='%d %m %Y',
    errors='coerce'
)


In [5]:


# --- ArrowMobile Data Processing ---
arrowmobile_df_processed = ArrowMobile_data_df.copy()
arrowmobile_df_processed.columns = [col.strip() for col in arrowmobile_df_processed.columns]
arrowmobile_df_processed = arrowmobile_df_processed.rename(columns={
    'CellNo': 'Cell_Phone_Number',
    'FirstName': 'First_Name',
    'LastName': 'Last_Name',
    'Area': 'Region',
    'SIMDate': 'SIM_Activation_Date'
})
arrowmobile_df_processed['Source_System_Name'] = 'ArrowMobile'

# Clean phone numbers
arrowmobile_df_processed['Cell_Phone_Number'] = (
    arrowmobile_df_processed['Cell_Phone_Number']
    .astype(str)
    .str.replace('+', '', regex=False)
    .str.strip()
)

# Parse SIM date
arrowmobile_df_processed['SIM_Activation_Date'] = pd.to_datetime(
    arrowmobile_df_processed['SIM_Activation_Date'].astype(str).str.strip(),
    format='%d %m %Y',
    errors='coerce'
)

# No DOB available
arrowmobile_df_processed['Date_Of_Birth'] = pd.NaT


# --- Ensure all dataframes have the same structure ---
required_columns = [
    'First_Name', 'Last_Name', 'Cell_Phone_Number',
    'Date_Of_Birth', 'Region', 'SIM_Activation_Date',
    'Source_System_Name'
]

vmobile_df_processed = vmobile_df_processed[required_columns]
bluemobile_df_processed = bluemobile_df_processed[required_columns]
arrowmobile_df_processed = arrowmobile_df_processed[required_columns]


### Merge all subscribers

In [ ]:

# --- Merge all subscribers ---
all_subscribers_df = pd.concat([vmobile_df_processed, bluemobile_df_processed, arrowmobile_df_processed],ignore_index=True)

print(all_subscribers_df.head(40))

      First_Name  Last_Name Cell_Phone_Number Date_Of_Birth            Region SIM_Activation_Date Source_System_Name
0       Sibonelo     Jiyane       27701302724    1965-05-31      Johannesburg          2024-01-10           V Mobile
1      Gugulethu    Dimbani       27702088773    1976-11-17          Pretoria          2024-11-15           V Mobile
2      Nokulunga    Mhlungu       27702970958    1977-03-11         West Rand          2025-04-16           V Mobile
3         Owethu   Khanyile       27703320064    1997-07-11    Port Elizabeth          2024-04-10           V Mobile
4        Lindsay      Blair       27704131178    1976-01-31         Cape Town          2024-09-22           V Mobile
5   Nomathalente      Shoba       27704753382    1962-09-06    Port Elizabeth          2024-08-06           V Mobile
6         Andrea   Calderon       27705752990    1981-09-06            Durban          2023-08-02           V Mobile
7          Henry      Smith       27706327368    1949-02-01     

### Master Record Logic Implementation

In [7]:

# --- Master Record Logic Implementation ---

# Initialize a DataFrame to hold the consolidated master records
consolidated_subscribers_df = pd.DataFrame(columns=required_columns + ['Is_Master_Record'])

# 1. Process V Mobile first (highest priority)
# All V Mobile records are considered master if they exist
consolidated_subscribers_df = pd.concat(
    [consolidated_subscribers_df, vmobile_df_processed.assign(Is_Master_Record=True)],
    ignore_index=True
)

# 2. Process BlueMobile for cell numbers not in consolidated_subscribers_df yet
bluemobile_unique = bluemobile_df_processed[
    ~bluemobile_df_processed['Cell_Phone_Number'].isin(consolidated_subscribers_df['Cell_Phone_Number'])
].copy()

# 3. Process ArrowMobile for cell numbers not in consolidated_subscribers_df yet
arrowmobile_unique = arrowmobile_df_processed[
    ~arrowmobile_df_processed['Cell_Phone_Number'].isin(consolidated_subscribers_df['Cell_Phone_Number'])
].copy()

# Combine unique BlueMobile and ArrowMobile records for comparison
combined_other_sources = pd.concat([bluemobile_unique, arrowmobile_unique], ignore_index=True)

# For duplicates within combined_other_sources, apply BlueMobile/latest SIM Activation Date rule
if not combined_other_sources.empty:
    # Set a custom sorting order for Source_System_Name (BlueMobile > ArrowMobile)
    source_order = pd.CategoricalDtype(
        ['BlueMobile', 'ArrowMobile'], ordered=True
    )
    combined_other_sources['Source_System_Name_Ordered'] = \
        combined_other_sources['Source_System_Name'].astype(source_order)

    combined_other_sources.sort_values(
        by=['Cell_Phone_Number', 'SIM_Activation_Date', 'Source_System_Name_Ordered'],
        ascending=[True, False, True], # Cell No ASC, SIM Date DESC, then BlueMobile (True for BlueMobile's lower category value)
        inplace=True
    )

    # Keep the first occurrence after sorting, which will be the master for this group
    other_sources_master = combined_other_sources.drop_duplicates(
        subset=['Cell_Phone_Number'], keep='first'
    ).copy()
    other_sources_master['Is_Master_Record'] = True

    # Drop the temporary sorting column
    other_sources_master.drop(columns=['Source_System_Name_Ordered'], inplace=True)

    # Add to consolidated_subscribers_df
    consolidated_subscribers_df = pd.concat([consolidated_subscribers_df, other_sources_master], ignore_index=True)



C:\Users\mini\AppData\Local\Temp\ipykernel_11464\420716923.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  consolidated_subscribers_df = pd.concat(


In [8]:

# Final Deduplication and Master Flagging
# This step ensures that even if initial V Mobile records had duplicates by Cell_Phone_Number,
# only one is kept and correctly marked as master.
# Sort to ensure V Mobile records (if any duplicates) are prioritized, then latest SIM.
final_subscriber_data_df = consolidated_subscribers_df.sort_values(
    by=[
        'Cell_Phone_Number',
        'Is_Master_Record',  # True masters first
        'SIM_Activation_Date', # Then latest SIM date
    ],
    # Custom key for Source_System_Name if needed for tie-breaking with identical SIM dates and Is_Master_Record status
    # This specifically puts V Mobile first, then BlueMobile, then ArrowMobile
    key=lambda x: x.map({'V Mobile': 0, 'BlueMobile': 1, 'ArrowMobile': 2})
    if x.name == 'Source_System_Name' else x,
    ascending=[True, False, False] # Cell No ASC, Master DESC, SIM Date DESC
).drop_duplicates(subset=['Cell_Phone_Number'], keep='first').reset_index(drop=True)


print("________________________________________________Final Consolidated Subscriber Data_________________________________________")
print(final_subscriber_data_df.head(10).to_string())
print(f"\nTotal unique subscribers in master data: {len(final_subscriber_data_df)}")

# Create a mapping from Cell_Phone_Number to Region for later use
subscriber_region_map = final_subscriber_data_df.set_index('Cell_Phone_Number')['Region'].to_dict()

________________________________________________Final Consolidated Subscriber Data_________________________________________
   First_Name  Last_Name Cell_Phone_Number Date_Of_Birth            Region SIM_Activation_Date Source_System_Name Is_Master_Record
0       Roger  Pritchard       27700408017    1998-12-17    Port Elizabeth          2024-12-23         BlueMobile             True
1  Sithembiso     Sabelo       27700498064    1961-01-02         Cape Town          2023-10-05         BlueMobile             True
2     Thabani      Phewa       27700536653    1960-10-20        Soshanguve          2025-06-03         BlueMobile             True
3      Donald       Lowe       27700594567           NaT        Ekurhuleni          2023-07-20        ArrowMobile             True
4    Sibonelo     Jiyane       27701302724    1965-05-31      Johannesburg          2024-01-10           V Mobile             True
5        Erin    Carlson       27701422836    1965-10-10        Soshanguve          2024-1

In [9]:
City_lookup_df

,CITY_ID,PROVINCE_NAME,CITY_NAME,ALTERNATIVE_CITY_NAME,CITY_LATITUDE,CITY_LONGITUDE,CITY_POPULATION
0,1,Eastern Cape,East London,NaN,"-32,98084","27,90841",756 507
1,2,Eastern Cape,Port Elizabeth,Gqeberha,"-33,96083","25,60217",1 330 500
2,3,Eastern Cape,Uitenhage,Kariega,"-33,76606","25,38848",198 800
3,4,Free State,Bloemfontein,Mangaung,"-29,12499","26,21998",618 817
4,5,Free State,Botshabelo,NaN,"-29,23688","26,73047",185 900
5,6,Free State,Sasolburg,NaN,"-27,76672","26,79769",114 000
6,7,Free State,Welkom,NaN,"-27,97445","26,73149",431 944
7,8,Gauteng,Benoni,NaN,"-26,15688","28,31894",487 700
8,9,Gauteng,Boksburg,NaN,"-26,24841","28,24638",348 100
9,10,Gauteng,Ekurhuleni,East Rand,"-26,14211","28,30644",4 262 130


In [10]:
# --- Usage Records Processing ---
usage_records_df_processed = usage_record_df.copy()

# Standardize Cell Number format
usage_records_df_processed.rename(columns={'MSISDN': 'Cell_Phone_Number'}, inplace=True)
usage_records_df_processed['Cell_Phone_Number'] = (
    usage_records_df_processed['Cell_Phone_Number']
    .astype(str)
    .str.replace('+', '', regex=False)
    .str.strip()
)

# Convert DATE_TIME to datetime objects
usage_records_df_processed['DATE_TIME'] = pd.to_datetime(
    usage_records_df_processed['USAGE_EVENT_DATE_TIME'],
    format='%d %m %Y %H:%M'
)

# Convert revenue strings like "10,28" → float
usage_records_df_processed['USAGE_EVENT_REVENUE'] = (
    usage_records_df_processed['USAGE_EVENT_REVENUE']
    .astype(str)
    .str.replace(',', '.', regex=False)
    .astype(float)
)


In [11]:

# Merge with usage event lookup to get human-readable event types
usage_records_df_processed = pd.merge(
    usage_records_df_processed,
    usage_event_lookup_df,
    on='USAGE_EVENT_TYPE_ID',
    how='left'
)

# Merge with City lookup to get location name
usage_records_df_processed = pd.merge(
    usage_records_df_processed,
    City_lookup_df[['CITY_ID', 'CITY_NAME']],
    left_on='USAGE_EVENT_CITY_ID',
    right_on='CITY_ID',
    how='left'
)


In [12]:

# Determine Week Start and End Dates
usage_records_df_processed['Week_Start_Date'] = (
    usage_records_df_processed['DATE_TIME']
    - pd.to_timedelta(usage_records_df_processed['DATE_TIME'].dt.weekday, unit='d')
).dt.normalize()

usage_records_df_processed['Week_End_Date'] = (
    usage_records_df_processed['Week_Start_Date'] + timedelta(days=6, hours=23, minutes=59, seconds=59)
)

print("\n________________________________________________Processed Usage Records (First 5 rows)_________________________________")
print(usage_records_df_processed.head())
print(f"\nTotal usage records: {len(usage_records_df_processed)}")



________________________________________________Processed Usage Records (First 5 rows)_________________________________
  Cell_Phone_Number USAGE_EVENT_DATE_TIME  USAGE_EVENT_CITY_ID  USAGE_EVENT_TYPE_ID  USAGE_EVENT_TRACKING_QUANTITY USAGE_EVENT_TRACKING_UNIT  USAGE_EVENT_BILLING_QUANTITY USAGE_EVENT_BILLING_UNIT  USAGE_EVENT_REVENUE           DATE_TIME        USAGE_EVENT_TYPE  CITY_ID     CITY_NAME Week_Start_Date       Week_End_Date
0       27745858892      18 07 2025 10:53                   10                    5                            257                   seconds                           257                  seconds                10.28 2025-07-18 10:53:00  on-net-call-per-second       10    Ekurhuleni      2025-07-14 2025-07-20 23:59:59
1       27759277187      15 07 2025 13:52                   10                    5                            241                   seconds                           241                  seconds                 7.23 2025-07-15 13:52:00  o

In [13]:
# --- Calculate Weekly Aggregates per Subscriber ---
# Identify call and SMS event types
call_event_types = [
    'landline-call-per-minute', 'on-net-call-per-minute',
    'on-net-call-per-second', 'other-mobile-call-per-minute',
    'other-mobile-call-per-second'
]
sms_event_types = ['on-net-sms', 'other-mobile-sms']

weekly_aggregates = usage_records_df_processed.groupby(
    ['Cell_Phone_Number', 'Week_Start_Date', 'Week_End_Date']
).agg(
    Total_Revenue=('USAGE_EVENT_REVENUE', 'sum'),
    Total_SMS_Count=('USAGE_EVENT_TYPE', lambda x: x.isin(sms_event_types).sum()),
    Total_Voice_Call_Count=('USAGE_EVENT_TYPE', lambda x: x.isin(call_event_types).sum())
).reset_index()


In [14]:

# Round Total_Revenue to 2 decimal places
weekly_aggregates['Total_Revenue'] = weekly_aggregates['Total_Revenue'].round(2)

# --- Identify Qualified Subscribers for each week ---
qualified_subscribers_weekly = weekly_aggregates[
    weekly_aggregates['Total_Revenue'] >= 30
].copy()

qualified_subscribers_weekly['Qualified'] = True

weekly_aggregates = pd.merge(
    weekly_aggregates,
    qualified_subscribers_weekly[['Cell_Phone_Number', 'Week_Start_Date', 'Qualified']],
    on=['Cell_Phone_Number', 'Week_Start_Date'],
    how='left'
).fillna({'Qualified': False})

# --- Join with Master Subscriber Data for Report Details ---
marketing_report_df = pd.merge(
    weekly_aggregates,
    final_subscriber_data_df[['Cell_Phone_Number', 'First_Name', 'Last_Name', 'Region']],
    on='Cell_Phone_Number',
    how='left'
)


In [15]:

# Filter only qualified subscribers
marketing_report_df = marketing_report_df[marketing_report_df['Qualified'] == True].copy()

# Select and reorder columns
marketing_report_df = marketing_report_df[[
    'Week_Start_Date', 'Week_End_Date', 'First_Name', 'Last_Name',
    'Cell_Phone_Number', 'Region', 'Total_Revenue',
    'Total_SMS_Count', 'Total_Voice_Call_Count'
]]

# Format dates as yyyymmdd
marketing_report_df['Reporting_Date_Start'] = marketing_report_df['Week_Start_Date'].dt.strftime('%Y%m%d')
marketing_report_df['Reporting_Date_End'] = marketing_report_df['Week_End_Date'].dt.strftime('%Y%m%d')

# Drop datetime objects if not needed
marketing_report_df.drop(columns=['Week_Start_Date', 'Week_End_Date'], inplace=True)

print("\n________________________________________________Weekly Marketing Report for Qualified Subscribers_______________________")
print(marketing_report_df.head(10).to_string())
print(f"\nTotal qalified subscriber records in report: {len(marketing_report_df)}")



________________________________________________Weekly Marketing Report for Qualified Subscribers_______________________
   First_Name  Last_Name Cell_Phone_Number            Region  Total_Revenue  Total_SMS_Count  Total_Voice_Call_Count Reporting_Date_Start Reporting_Date_End
0       Roger  Pritchard       27700408017    Port Elizabeth         313.36                0                      11             20250714           20250720
1  Sithembiso     Sabelo       27700498064         Cape Town         262.96                0                      10             20250714           20250720
2     Thabani      Phewa       27700536653        Soshanguve         306.60                1                      12             20250714           20250720
3      Donald       Lowe       27700594567        Ekurhuleni         193.56                0                       7             20250714           20250720
4    Sibonelo     Jiyane       27701302724      Johannesburg         167.20                0 

In [16]:
# save to CSV
#marketing_report_df.to_csv('Data/Weekly_Marketing_Report.csv', index=False)


In [ ]:
#Code identifing Cell Phone Numbers with more than 30 revenue in a week
high_revenue_subscribers = marketing_report_df[marketing_report_df['Total_Revenue'] > 60] # Changed to > R60 to match the requirement over R30 per week
print("\n________________________________________________High Revenue Subscribers (Revenue > 30)_______________________")

# List out unique high revenue subscribers
unique_high_revenue_subscribers = high_revenue_subscribers['First_Name'].unique()
print(unique_high_revenue_subscribers)

# Total unique high revenue subscribers
print(f"\nTotal qualifying subscribers : {len(unique_high_revenue_subscribers)}")


________________________________________________High Revenue Subscribers (Revenue > 30)_______________________
['Roger' 'Sithembiso' 'Thabani' 'Donald' 'Sibonelo' 'Erin' 'Sibongile'
 'Gugulethu' 'Margaret' 'Lewis' 'Nokulunga' 'Bandile' 'Luyanda' 'Owethu'
 'Nomasonto' 'Busisiwe' 'Lindsay' 'Johw' 'Jill' 'Andile' 'Nomathalente'
 'Andrea' 'Zinhle' 'Thembeka' 'Henry' 'Nozizwe' 'Bhekisisa' 'Michael'
 'Siphesihle' 'Sibusisiwe' 'David' 'Thembile' 'Nicole' 'Linda'
 'Sihawukele' 'Silondile' 'Mlungisi' 'Carlos' 'Eric' 'Thabisa' 'Londiwe'
 'Ross' 'Nomcebo' 'Lungile' 'Vusumuzi' 'Dumisani' 'Mitchell' 'Joan'
 'Thando' 'Ndumiso' 'William' 'Amahle' 'Mandla' 'Julia' 'Simphiwe' 'Gugu'
 'Thuthukile' 'Patrick' 'Sandile' 'Timothy' 'Guy' 'John' 'Jason'
 'Thembisile' 'Ashley' 'Gemma' 'Nomthandazo' 'Hannah' 'Mnqo'
 'SimphiweyiNkosi' 'Sandra' 'Tomy' 'Janice' 'Katy' 'Lwandile' 'Lynda'
 'Danielle' 'Zakhele' 'Thandazile' 'Nomy' 'Thembekile' 'Vanessa' 'Zenzile'
 'Jabulani' 'Ruth' 'Janw' 'Antony' 'Bhekizizwe' 'Debr